In [1]:
import pandas as pd

In [21]:
df_my_results = pd.read_csv("../BF_C2DL-HSC_results.csv")

In [22]:
df_my_results = df_my_results[df_my_results['competitor'] == "CALT-US"]

In [24]:
df_my_results = df_my_results[df_my_results['campaign'] == 2]

In [25]:
df_my_results

,competitor,campaign,image_key,label,jaccard_score,image_average,campaign_average,overall_competitor_average
2511,CALT-US,2,02_1585.tif,85,0.670213,0.848265,0.852573,0.871878
2512,CALT-US,2,02_1585.tif,113,0.896226,0.848265,0.852573,0.871878
2513,CALT-US,2,02_1585.tif,132,0.850394,0.848265,0.852573,0.871878
2514,CALT-US,2,02_1585.tif,137,0.860841,0.848265,0.852573,0.871878
2515,CALT-US,2,02_1585.tif,153,0.901099,0.848265,0.852573,0.871878
...,...,...,...,...,...,...,...,...
2815,CALT-US,2,02_0287.tif,7,0.868159,0.877262,0.852573,0.871878
2816,CALT-US,2,02_0287.tif,634,0.852507,0.877262,0.852573,0.871878
2817,CALT-US,2,02_0287.tif,635,0.875339,0.877262,0.852573,0.871878
2818,CALT-US,2,02_0046.tif,1,0.920330,0.915178,0.852573,0.871878


In [32]:
df_my_results['normalized_label'] = df_my_results.groupby('image_key').cumcount() + 1

/var/folders/hd/b12972f51wdgmbmslytxnhvc0000gn/T/ipykernel_21042/1148820858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_my_results['normalized_label'] = df_my_results.groupby('image_key').cumcount() + 1


In [33]:
df_my_results

,competitor,campaign,image_key,label,jaccard_score,image_average,campaign_average,overall_competitor_average,normalized_label
2511,CALT-US,2,02_1585.tif,85,0.670213,0.848265,0.852573,0.871878,1
2512,CALT-US,2,02_1585.tif,113,0.896226,0.848265,0.852573,0.871878,2
2513,CALT-US,2,02_1585.tif,132,0.850394,0.848265,0.852573,0.871878,3
2514,CALT-US,2,02_1585.tif,137,0.860841,0.848265,0.852573,0.871878,4
2515,CALT-US,2,02_1585.tif,153,0.901099,0.848265,0.852573,0.871878,5
...,...,...,...,...,...,...,...,...,...
2815,CALT-US,2,02_0287.tif,7,0.868159,0.877262,0.852573,0.871878,2
2816,CALT-US,2,02_0287.tif,634,0.852507,0.877262,0.852573,0.871878,3
2817,CALT-US,2,02_0287.tif,635,0.875339,0.877262,0.852573,0.871878,4
2818,CALT-US,2,02_0046.tif,1,0.920330,0.915178,0.852573,0.871878,1


In [26]:
## parse the SEG_log

In [30]:
with open("../data/inputs-2020-07/BF-C2DL-HSC/CALT-US/02_RES/SEG_log.txt", 'r') as seg_log_file:
    seg_log = seg_log_file.readlines()

In [40]:
import re
import pandas as pd
import io # Needed if input is a list of strings, not a file path

def parse_jaccard_log(file_content):
    """
    Parses log file content with T/Z headers and GT_label/J data lines
    into a pandas DataFrame.

    Args:
        file_content (list or file-like object):
            Either a list of strings (each string is a line from the file)
            or an open file-like object (e.g., opened with open()).

    Returns:
        pandas.DataFrame: A DataFrame with columns:
                          'image_key', 'label', 'original_jaccard_score'.
                          Returns an empty DataFrame if no data is found.
    """
    # Regex patterns to match the lines
    # Header: ---T=46 Z=0---
    header_pattern = re.compile(r'^-+T=(\d+)\s+Z=\d+-+$')
    # Data: GT_label=1 J=0.92033
    data_pattern = re.compile(r'^GT_label=(\d+)\s+J=([\d.]+)$')

    parsed_data = []
    current_image_key = None

    # Determine if input is a list of lines or a file-like object
    if isinstance(file_content, list):
        lines = file_content
    else:
        # Assuming file_content is a file-like object (e.g., from open())
        lines = file_content # File objects are iterable line by line

    for line in lines:
        line = line.strip() # Remove leading/trailing whitespace and newline

        # Try matching header pattern
        header_match = header_pattern.match(line)
        if header_match:
            t_value = int(header_match.group(1))
            # Format the image key as requested: 02_TTT.tif
            current_image_key = f"02_{t_value:04d}.tif"
            # print(f"DEBUG: Found header, setting image_key to {current_image_key}") # Optional debug
            continue # Move to the next line

        # Try matching data pattern
        data_match = data_pattern.match(line)
        if data_match:
            if current_image_key is None:
                # This shouldn't happen in a well-formed file after the first header
                print(f"Warning: Found data line before a header: {line}. Skipping.")
                continue

            label = int(data_match.group(1))
            jaccard_score = float(data_match.group(2))

            parsed_data.append({
                'image_key': current_image_key,
                'label': label,
                'original_jaccard_score': jaccard_score
            })
            # print(f"DEBUG: Added data: {parsed_data[-1]}") # Optional debug
            continue # Move to the next line

    # Create DataFrame from the list of dictionaries
    if not parsed_data:
        print("Warning: No data rows were parsed.")
        return pd.DataFrame(columns=['image_key', 'label', 'original_jaccard_score'])
    else:
        return pd.DataFrame(parsed_data)

seg_log_filepath = "../data/inputs-2020-07/BF-C2DL-HSC/CALT-US/02_RES/SEG_log.txt"
df_from_file = None
try:
    with open(seg_log_filepath, 'r') as seg_log_file:
        df_from_file = parse_jaccard_log(seg_log_file)
        print(f"--- DataFrame parsed from file '{seg_log_filepath}' ---")
        print(df_from_file)
except FileNotFoundError:
    print(f"Error: File not found at {seg_log_filepath}")

--- DataFrame parsed from file '../data/inputs-2020-07/BF-C2DL-HSC/CALT-US/02_RES/SEG_log.txt' ---
       image_key  label  original_jaccard_score
0    02_0046.tif      1                0.920330
1    02_0046.tif      2                0.910026
2    02_0056.tif      1                0.791262
3    02_0056.tif      2                0.851429
4    02_0287.tif      1                0.913043
..           ...    ...                     ...
304  02_1748.tif    152                0.870432
305  02_1748.tif    153                0.865031
306  02_1748.tif    154                0.893082
307  02_1748.tif    155                0.826425
308  02_1748.tif    156                0.920455

[309 rows x 3 columns]


In [41]:
df_from_file

,image_key,label,original_jaccard_score
0,02_0046.tif,1,0.920330
1,02_0046.tif,2,0.910026
2,02_0056.tif,1,0.791262
3,02_0056.tif,2,0.851429
4,02_0287.tif,1,0.913043
...,...,...,...
304,02_1748.tif,152,0.870432
305,02_1748.tif,153,0.865031
306,02_1748.tif,154,0.893082
307,02_1748.tif,155,0.826425


In [42]:
merged_df = pd.merge(
    df_my_results,
    df_from_file,
    how='left',
    left_on=['image_key', 'normalized_label'], # Keys in my_df
    right_on=['image_key', 'label']          # Corresponding keys in parsed_df
)

In [43]:
merged_df

,competitor,campaign,image_key,label_x,jaccard_score,image_average,campaign_average,overall_competitor_average,normalized_label,label_y,original_jaccard_score
0,CALT-US,2,02_1585.tif,85,0.670213,0.848265,0.852573,0.871878,1,1,0.670213
1,CALT-US,2,02_1585.tif,113,0.896226,0.848265,0.852573,0.871878,2,2,0.896226
2,CALT-US,2,02_1585.tif,132,0.850394,0.848265,0.852573,0.871878,3,3,0.850394
3,CALT-US,2,02_1585.tif,137,0.860841,0.848265,0.852573,0.871878,4,4,0.860841
4,CALT-US,2,02_1585.tif,153,0.901099,0.848265,0.852573,0.871878,5,5,0.901099
...,...,...,...,...,...,...,...,...,...,...,...
304,CALT-US,2,02_0287.tif,7,0.868159,0.877262,0.852573,0.871878,2,2,0.868159
305,CALT-US,2,02_0287.tif,634,0.852507,0.877262,0.852573,0.871878,3,3,0.852507
306,CALT-US,2,02_0287.tif,635,0.875339,0.877262,0.852573,0.871878,4,4,0.875339
307,CALT-US,2,02_0046.tif,1,0.920330,0.915178,0.852573,0.871878,1,1,0.920330


In [45]:
import numpy as np

In [46]:
# Define a small tolerance for floating point comparison
tolerance = 1e-6  # Adjust as needed (e.g., 1e-8 for higher precision)

# Calculate the absolute difference
# Use .fillna() to handle potential NaNs during subtraction, treating NaN as different from any number.
# If a score is NaN and the other is not, the difference will be large (or NaN), satisfying > tolerance.
# If both are NaN, fillna won't help directly, but the result of NaN - NaN is NaN. abs(NaN) is NaN. NaN > tolerance is False.
# This correctly excludes rows where *both* scores are NaN from being flagged as different.
abs_diff = (merged_df['jaccard_score'].fillna(-np.inf) -
            merged_df['original_jaccard_score'].fillna(-np.inf)).abs()


# Create a boolean mask: True where the difference > tolerance OR where one is NaN and the other isn't
# The fillna approach above handles the NaN vs non-NaN case implicitly.
# We just need the check against tolerance.
mask_differ = abs_diff > tolerance

# Filter the DataFrame using the mask
differing_scores_df = merged_df[mask_differ]

print(f"--- Rows where jaccard_score and original_jaccard_score differ (tolerance={tolerance}) ---")
if differing_scores_df.empty:
    print("No rows found where scores differ significantly.")
else:
    print(differing_scores_df)

--- Rows where jaccard_score and original_jaccard_score differ (tolerance=1e-06) ---
    competitor  campaign    image_key  label_x  jaccard_score  image_average  \
151    CALT-US         2  02_1748.tif      275       0.835088       0.846263   
157    CALT-US         2  02_1748.tif      315       0.783262       0.846263   
194    CALT-US         2  02_1748.tif      382       0.831832       0.846263   
294    CALT-US         2  02_1748.tif      571       0.895522       0.846263   

     campaign_average  overall_competitor_average  normalized_label  label_y  \
151          0.852573                    0.871878                 5        5   
157          0.852573                    0.871878                11       11   
194          0.852573                    0.871878                48       48   
294          0.852573                    0.871878               148      148   

     original_jaccard_score  
151                0.439926  
157                0.460277  
194                0.35